In [75]:
from langchain_ollama.llms import OllamaLLM
model = OllamaLLM(model="llama3:8b")

print(model.invoke("who are you?"))

I am LLaMA, an AI assistant developed by Meta AI that can understand and respond to human input in a conversational manner. I'm not a human, but a computer program designed to simulate conversation and answer questions to the best of my ability based on my training data.

My primary function is to provide information, answer questions, and engage in natural-sounding conversations with humans. I can discuss topics like science, history, entertainment, and more. I can also generate text, translate languages, and even create simple stories or dialogues.

I'm constantly learning and improving my responses based on the interactions I have with users like you. This means that over time, I'll become better at understanding what you want to talk about and providing helpful and accurate information.

So, who am I? I'm a machine learning model designed to help people communicate and find answers in a fun and engaging way!


In [76]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

In [77]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

local_embeddings = OllamaEmbeddings(model="nomic-embed-text")

vectorstore = Chroma.from_documents(documents=all_splits, embedding=local_embeddings)

In [78]:
question = "What are the approaches to Task Decomposition?"
docs = vectorstore.similarity_search(question)
len(docs)

4

In [80]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    "Summarize the main themes in these retrieved docs: {docs}"
)


# Convert loaded documents into strings by concatenating their content
# and ignoring metadata
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain = {"docs": format_docs} | prompt | model | StrOutputParser()

question = "What are the approaches to Task Decomposition?"

docs = vectorstore.similarity_search(question)

chain.invoke(docs)

'It seems like there\'s a bit of repetition in these documents! However, I\'ll summarize the main themes for you:\n\nThe main theme is that Task Decomposition can be achieved through three methods:\n\n1. **Large Language Models (LLMs)**: Using simple prompts like "Steps for XYZ" or "What are the subgoals for achieving XYZ?" to break down a task into smaller steps.\n2. **Task-specific instructions**: Providing specific guidance, such as "Write a story outline," to help decompose a task.\n3. **Human inputs**: Involving humans in the process to provide input and guide the decomposition of a task.\n\nThese methods can be used separately or in combination to effectively break down complex tasks into manageable parts.'

In [83]:
from langchain_core.runnables import RunnablePassthrough

RAG_TEMPLATE = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

<context>
{context}
</context>

Answer the following question:

{question}"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

chain = (
    RunnablePassthrough.assign(context=lambda input: format_docs(input["context"]))
    | rag_prompt
    | model
    | StrOutputParser()
)

question = "What are the approaches to Task Decomposition?"

docs = vectorstore.similarity_search(question)

# Run
chain.invoke({"context": docs, "question": question})

'According to the retrieved context, there are three approaches to Task Decomposition: (1) using LLM with simple prompting, (2) using task-specific instructions, and (3) with human inputs.'

In [84]:
retriever = vectorstore.as_retriever()

qa_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | model
    | StrOutputParser()
)

In [86]:
question = "What are the approaches to Task Decomposition?"

qa_chain.invoke(question)

'According to the provided context, there are three approaches to Task Decomposition: (1) by using Large Language Models (LLM) with simple prompting, (2) by using task-specific instructions, and (3) with human inputs.'